In [2]:
from numpy.random import rand
import numpy as np
from numpy.random import choice
from numpy import asarray
from numpy import clip
from numpy import argmin
from numpy import min
from numpy import around
from matplotlib import pyplot
import random
from numpy.random import choice
import copy
import time
start_time = time.time()

def generate_fragment_sizes(num_fragments, c10):
    return [random.randint(10, 20 * c10) for _ in range(num_fragments)]


def calculation(solution):
    cost1 = 0 # partial cost
    cost2 = 0 # direct cost
    cost = 0
    for j1 in range(m):
        for j2 in range(m):
            cost1 = cost1 + (frdep[j1,j2] + uc[solution[j1],solution[j2]])
    for j in range(m):        
        for q in range(n):   # partialco
            r1 = 0
            for k in range(l):
                r1 = r1 + (frequency[solution[j],k] * trfr[k,j])
                cost2 = cost2 + (uc[solution[j],q] * r1)
    cost = cost1 + cost2
    return cost

def crossover(x,y,cr = 0.8):
    R = random.randrange(0,1)
    if R < cr:
        cutp1 = random.randrange(1,m)
        cutp2 = random.randrange(1,m)
        for i in range(cutp1):
            x[i],y[i] = y[i], x[i]
        for i in range(cutp2,m):
            x[i],y[i] = y[i], x[i]
    
    return x,y

# def crossover(x,y,cr = 0.8):
#     x1 = copy.deepcopy(x)
#     y1= copy.deepcopy(y)
#     R = random.randrange(0,1)
#     if R < cr:
#         cutp1 = random.randrange(1,m)
#         cutp2 = random.randrange(1,m)
#         for i in range(cutp1):
#             x[i],y[i] = y[i], x[i]
#         for i in range(cutp2,m):
#             x[i],y[i] = y[i], x[i]
    
#     if calculation(x) < calculation(x1):
#         p = x
#     else:
#         p = x1
#     if calculation(y) < calculation(y1):
#         q = y
#     else:
#         q = y1
#     return p,q

def mutation(X,Y):
    if calculation(X) < calculation(Y):
            Z = X
    elif calculation(Y) < calculation(X):
        Z = Y
    else:
        rand = np.random.randint(0,2)
        if rand == 0:
            Z = X
        else:
            Z = Y
    return Z

c = 10
PopSize = 200

m = 5 # number of fragments
n = 5# number of sites
l = 5 # number of transactions
PM = 0.4
fragment_size = generate_fragment_sizes(m,c)[0]
K = 500 # number of iterations
GenSize = 200
RPT = 0.7 #Probability of transaction requested at a site
APF = 0.4 #Probability of fragment accessed by transaction
APFS = 0.025
sol = [[0]*m]*PopSize
# Solution initialization
for i in range(1,PopSize):
    for j in range(1,m):
        site = random.randint(0,n-1)
        sol[i][j] = site
sol = np.array(sol)
sol2 = copy.deepcopy(sol)
site_capacity = m*fragment_size
UCN = random.uniform(0,1) # random number between 0 and 1
uc = np.random.uniform(0, UCN * n, size=(n, n))
uc = (uc + uc.T) / 2

#uc = np.random.randint(UCN,UCN*n,size=(n,n))
#uc = (uc + uc.T)/2
numberlist = [i for i in range(n)]
weights = [RPT]*n
x = random.choices(numberlist,weights = weights, k = n)




# Generating frequency matrix
freq = [[0 for _ in range(l)] for _ in range(n)]
for i in range(n):
    for j in range(l):
        flag = random.randint(0, 1)
        if flag == 1:
            freq[i][j] = random.randint(1, 1000)*RPT
        else:
            freq[i][j] = 0
frequency = np.array(freq)



# Generating Q matrix
Q = [[[0]*l]*m]*m
for i in range(m):
    for j in range(m):
        for k in range(l):
            flag = random.randint(0,1) 
            if flag == 0:
                Q[i][j][k] = random.randint(0,fragment_size) * APFS

trfr = [[0]*m]*l
for i in range(l):
    for j in range(m):
        flag = random.randint(0,1)
        if flag == 1:
            trfr[i][j] = random.randrange(1,fragment_size)*APF
        else:
            trfr[i][j] = 0
trfr = np.array(trfr)
stfr = frequency @ trfr
qfr = np.zeros(shape=(l,m,m))
for i in range(l):
    for j1 in range(m):
        for j2 in range(m):
            qfr[i,j1,j2] = random.randrange(0,fragment_size-1)
frdep = np.zeros(shape=(m,m))
for i in range(m):
    for j in range(m):
        frdep[i,j] = np.sum(qfr[:,i,j])
        
def VNS(X):
    # neighbour structure will be a matrix of dimension 1Xm. where m = no of fragments = 20
    for k in range(10):
        Y = X
        for p in range(5):
            for q in range(3): # changing 3 sites randomly
                fragment = np.random.randint(0,m)
                site = np.random.randint(0,n)
                Z = Y
                Z[fragment] = site
            if calculation(Z) < calculation(Y):
                Y = Z
        if calculation(Y) < calculation(X):
            X = Y
    return X
        
import numpy as np

problem_size = 5
swarm_size = 50
number_of_iterations = 200

swarm = np.random.randint(0, problem_size, size=(swarm_size, problem_size))

# Initialize the personal best and the global best
population = sol
personal_best = population.copy()
global_best = personal_best[np.argmin(calculation(personal_best))]

# Run the hybrid PSO-GA algorithm for the specified number of iterations
for i in range(number_of_iterations):

    # Update the swarm velocity
    for j in range(swarm_size):
        swarm[j] = (swarm[j].astype('float64') + (personal_best[j] - swarm[j]) * np.random.random() + (global_best.astype('float64') - swarm[j]) * np.random.random()).astype('int32')

    # Update the swarm position
    for j in range(swarm_size):
        swarm[j] = np.clip(swarm[j], 0, problem_size - 1)

    # Update the personal best
    for j in range(swarm_size):
        if calculation(swarm[j]) < calculation(personal_best[j]):
            personal_best[j] = swarm[j].copy()

    # Update the global best
    global_best = personal_best[np.argmin(calculation(personal_best))]

    # Perform crossover and mutation on the population
    for j in range(0,PopSize,2):

        # Select one individual (row) from the population
        parent1 = np.random.choice(population.flatten(), 1)[0]
        parent2 = np.random.choice(population.flatten(), 1)[0]

        # Check if the parent variables are scalars
        if not isinstance(parent1, np.ndarray) or parent1.ndim == 0:
            parent1 = np.array([parent1])
        if not isinstance(parent2, np.ndarray) or parent2.ndim == 0:
            parent2 = np.array([parent2])

        crossover_point = np.random.randint(0, problem_size)

        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

        population[j] = child1
        population[j + 1] = child2

    for j in range(PopSize):
        mutation_point = np.random.randint(0, problem_size)
        population[j][mutation_point] = np.random.randint(0, problem_size)
    
# Return the best solution found
global_best = VNS(global_best)
print(global_best)
print(calculation(global_best)/(10**6))
print(time.time() - start_time)

[2 3 4 1 0]
0.5488065826172859
6.709619045257568


In [3]:
from numpy.random import rand
import numpy as np
from numpy.random import choice
from numpy import asarray
from numpy import clip
from numpy import argmin
from numpy import min
from numpy import around
from matplotlib import pyplot
import random
from numpy.random import choice
import copy
start_time = time.time()

def generate_fragment_sizes(num_fragments, c10):
    return [random.randint(10, 20 * c10) for _ in range(num_fragments)]


def calculation(solution):
    cost1 = 0 # partial cost
    cost2 = 0 # direct cost
    cost = 0
    for j1 in range(m):
        for j2 in range(m):
            cost1 = cost1 + (frdep[j1,j2] + uc[solution[j1],solution[j2]])
    for j in range(m):        
        for q in range(n):   # partialco
            r1 = 0
            for k in range(l):
                r1 = r1 + (frequency[solution[j],k] * trfr[k,j])
                cost2 = cost2 + (uc[solution[j],q] * r1)
    cost = cost1 + cost2
    return cost
    
def crossover(x,y,cr = 0.8):
    R = random.randrange(0,1)
    if R < cr:
        cutp1 = random.randrange(1,m)
        cutp2 = random.randrange(1,m)
        for i in range(cutp1):
            x[i],y[i] = y[i], x[i]
        for i in range(cutp2,m):
            x[i],y[i] = y[i], x[i]
    
    return x,y

# def crossover(x,y,cr = 0.8):
#     x1 = copy.deepcopy(x)
#     y1= copy.deepcopy(y)
#     R = random.randrange(0,1)
#     if R < cr:
#         cutp1 = random.randrange(1,m)
#         cutp2 = random.randrange(1,m)
#         for i in range(cutp1):
#             x[i],y[i] = y[i], x[i]
#         for i in range(cutp2,m):
#             x[i],y[i] = y[i], x[i]
    
#     if calculation(x) < calculation(x1):
#         p = x
#     else:
#         p = x1
#     if calculation(y) < calculation(y1):
#         q = y
#     else:
#         q = y1
#     return p,q


def mutation(X,Y):
    if calculation(X) < calculation(Y):
            Z = X
    elif calculation(Y) < calculation(X):
        Z = Y
    else:
        rand = np.random.randint(0,2)
        if rand == 0:
            Z = X
        else:
            Z = Y
    return Z

def VNS(X):
    # neighbour structure will be a matrix of dimension 1Xm. where m = no of fragments = 20
    for k in range(10):
        Y = X
        for p in range(5):
            for q in range(3): # changing 3 sites randomly
                fragment = np.random.randint(0,m)
                site = np.random.randint(0,n)
                Z = Y
                Z[fragment] = site
            if calculation(Z) < calculation(Y):
                Y = Z
        if calculation(Y) < calculation(X):
            X = Y
    return X
                
        
cost_values = []
for i in range(1):
    c = 10
    PopSize = 200
    m = 5 # number of fragments
    n = 5 # number of sites
    l = 5 # number of transactions
    PM = 0.4
    fragment_size = generate_fragment_sizes(m,c)[0]
    K = 200 # number of iterations
    GenSize = 200
    RPT = 0.7 #Probability of transaction requested at a site
    APF = 0.4 #Probability of fragment accessed by transaction
    APFS = 0.025
    sol = [[0]*m]*PopSize
    # Solution initialization
    for i in range(1,PopSize):
        for j in range(1,m):
            site = random.randint(0,n-1)
            sol[i][j] = site
    sol = np.array(sol)
    sol2 = copy.deepcopy(sol)
    site_capacity = m*fragment_size
    UCN = random.uniform(0,1) # random number between 0 and 1
    uc = np.random.uniform(0, UCN * n, size=(n, n))
    uc = (uc + uc.T) / 2

    #uc = np.random.randint(UCN,UCN*n,size=(n,n))
    #uc = (uc + uc.T)/2
    numberlist = [i for i in range(n)]
    weights = [RPT]*5
    x = random.choices(numberlist,weights = weights, k = n)




    # Generating frequency matrix
    freq = [[0 for _ in range(l)] for _ in range(n)]
    for i in range(n):
        for j in range(l):
            flag = random.randint(0, 1)
            if flag == 1:
                freq[i][j] = random.randint(1, 1000)*RPT
            else:
                freq[i][j] = 0
    frequency = np.array(freq)



    # Generating Q matrix
    Q = [[[0]*l]*m]*m
    for i in range(m):
        for j in range(m):
            for k in range(l):
                flag = random.randint(0,1) 
                if flag == 0:
                    Q[i][j][k] = random.randint(0,fragment_size) * APFS

    trfr = [[0]*m]*l
    for i in range(l):
        for j in range(m):
            flag = random.randint(0,1)
            if flag == 1:
                trfr[i][j] = random.randrange(1,fragment_size)*APF
            else:
                trfr[i][j] = 0
    trfr = np.array(trfr)
    stfr = frequency @ trfr
    qfr = np.zeros(shape=(l,m,m))
    for i in range(l):
        for j1 in range(m):
            for j2 in range(m):
                qfr[i,j1,j2] = random.randrange(0,fragment_size-1)
    frdep = np.zeros(shape=(m,m))
    for i in range(m):
        for j in range(m):
            frdep[i,j] = np.sum(qfr[:,i,j])

    # Differential Evolution
    COST = [calculation(solution) for solution in sol]
    newcost = [(max(COST)-COST[i]) for i in range(PopSize)]
    for i in range(5): # number of iteration
        for j in range(PopSize): # for all solution in sol
            candidates = [candidate for candidate in range(PopSize) if candidate != j]
            R1, R2, R3 = sol[choice(candidates, 3, replace=False)]
            A1,B1 = crossover(R2,R3)
            C = mutation(A1,B1)
            A2,B2 = crossover(R1,C)
            C = mutation(A2,B2)
            A3,B3 = crossover(sol[j],C)
            C = mutation(A3,B3)
            D = VNS(C)
            if calculation(D) < calculation(sol[j]):
                sol[j] = D


    #printing the solution
    final_cost = [calculation(i) for i in sol]
    

print("final solution :", sol[final_cost.index(min(final_cost))] )
print("cost of solution : ",min(final_cost)/(10**6))
print(time.time() - start_time)

final solution : [4 4 4 4 3]
cost of solution :  0.0764175121648756
24.934153079986572
